# 게임 리뷰 데이터 추출하기

## minimap 에서 리뷰를 추출해오는 함수 생성

In [1]:
def get_review(game):
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.common.exceptions import TimeoutException
    from selenium.webdriver.common.keys import Keys
    import pyperclip
    import time

    # Chrome 드라이버 설정
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)

    # 게임 리뷰 데이터를 저장할 reviews 리스트 생성
    reviews = []

    try:
        # 웹페이지 URL
        url = f"https://minimap.net/game/{game}/review?l=kr"

        # 웹페이지 로드
        driver.get(url)

        # 로그인 버튼 찾기 및 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".C-button .c-button-item.c-color-primary"))  # 실제 로그인 버튼의 CSS 선택자로 변경 필요        
        )
        login_button.click()

        login_button2 = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".pop-sign-up .button-w .login-button-w .login-button"))  # 실제 로그인 버튼의 CSS 선택자로 변경 필요
        )
        login_button2.click()

        # 로그인 팝업 창이 나타날 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#popupWrapper .popup-wrap-background .popup-dialog-wrap"))  # 실제 팝업 창의 CSS 선택자로 변경 필요
        )

        # 로그인 정보 입력
        id_field = driver.find_element(By.ID, "sign-in-email-input")  # 실제 사용자 이름 필드의 ID로 변경 필요
        pw_field = driver.find_element(By.ID, "sign-in-password-input")  # 실제 비밀번호 필드의 ID로 변경 필요

        pyperclip.copy("lhe9614@gmail.com")
        id_field.send_keys(Keys.CONTROL, 'v')

        pyperclip.copy("!Moon@126419#")
        pw_field.send_keys(Keys.CONTROL, 'v')

        time.sleep(2)

        # 로그인 제출 버튼 클릭
        submit_button = driver.find_element(By.CSS_SELECTOR, ".pop-sign-in .button-w .C-round-button")  # 실제 제출 버튼의 CSS 선택자로 변경 필요
        submit_button.click()

        # 로그인 완료 대기 (예: 특정 요소가 사라질 때까지)
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "#popupWrapper .popup-wrap-background .popup-dialog-wrap"))
        )

        # 로그인 완료 후 페이지가 완전히 로드될 때까지 대기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-list"))
        )

        # 리뷰 페이지로 다시 이동
        driver.get(url)

        # 리뷰 컨테이너 요소 찾기
        review_container = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-list"))
        )

        # 컨테이너 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 스크롤 반복해서 내리기
        ## 최대 반복 수 설정
        max_cnt = 100

        ## 이전 높이 계산
        old_height = driver.execute_script("return document.body.scrollHeight")

        cnt = 0
        while True:
            ## 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            ## 대기
            time.sleep(0.5)
            
            ## 새로운 높이 계산
            new_height = driver.execute_script("return document.body.scrollHeight")
            ## 더 이상 스크롤되지 않으면 종료
            if new_height == old_height:
                break  
            ## 새로운 높이 계산 값을 이전 높이 변수에 저장장 
            old_height = new_height

            ## 카운트 계산
            if cnt == max_cnt:
                break 
            cnt += 1

        # 모든 "post-content" 클래스를 가진 div 요소 찾기
        post_contents = driver.find_elements(By.CLASS_NAME, "post-content")
        
        # 각 "post-content" div의 텍스트 내용 추출
        for post in post_contents:
            review_text = post.text.strip()  # div 내의 모든 텍스트를 가져옴
            reviews.append(review_text)

    except Exception as e:
        print(f"오류 발생, {e}")

    finally:
        # 브라우저 종료
        driver.quit()
    
    return reviews

## game_list 에 해당하는 게임들의 리뷰 추출

In [ ]:
import pandas as pd

# 게임의 리뷰를 가져오기 위한 game_list 라는 변수들의 list 생성
game_list = {
    ('쿠키런킹덤', 'com-devsisters-ck'),
    ('던전앤파이터', 'dungeon-and-fighters'),
    ('메이플스토리', 'maplestory'),
    ('월드오브워크래프트', 'world-of-warcraft'),
    ('블레이드앤소울', 'blade-soul'),
    ('마비노기', 'mabinogi'),
    ('리그오브레전드', 'league-of-legends'),
    ('펜타스톰', 'arena-of-valor'),
    ('펜타스톰', 'com-netmarble-penta-gm'),
    ('도타2', 'dota-2'),
    ('브롤스타즈', 'com-supercell-brawlstars'),
    ('서든어택', 'sudden-attack'),
    ('배틀그라운드', 'playerunknowns-battlegrounds'),
    ('배틀그라운드', 'com-pubg-krmobile'),
    ('오버워치', 'overwatch'),
    ('발로란트', 'valorant'),
    ('클래시오브클랜', 'com-supercell-clashofclans'),
    ('클래시로얄', 'com-supercell-clashroyale-gm'),
    ('스타크래프트', 'starcraft'),
    ('피파온라인', 'fifa-online-4'),
    ('피파23', 'ea-sports-fifa-23'),
    ('캔디크러쉬사가', 'com-king-candycrushsaga'),
    ('마인크래프트', 'minecraft'),
    ('로블록스', 'roblox'),
    ('로블록스', 'com-roblox-client-gm'),
    ('카트라이더', 'kartrider'),
    ('테일즈런너', 'tales-runner'),
    ('크레이지아케이드', 'crazy-arcade'),
    ('모여봐요동물의숲', 'animal-crossing-new-horizons'),
    ('모두의마블', 'com-cjenm-modoomarblekakao-gm'),
    ('포켓몬스터', 'pokemon-scarlet')
}

# 모든 리뷰를 저장할 리스트 생성
all_reviews = []

for game_name, game_id in game_list:
    game_reviews = get_review(game_id)
    
    for review in game_reviews:
        all_reviews.append({
            "Game": game_name,
            "Review": review
        })

# 리뷰 데이터를 데이터프레임으로 변환
game_reviews_df = pd.DataFrame(all_reviews)

# 데이터프레임 출력
print(game_reviews_df)

# CSV 파일로 저장
game_reviews_df.to_csv('game_reviews.csv', index=False)



      Game                                             Review
0     로블록스                     이 게임 하나의 모든 장르에 게임이 있어서 정말 추천함
1     로블록스                                         진짜 할 거 없어요
2     로블록스  게임이라기 보단 게임들이 모여있는 플렛폼에 더 가깝다.\n하지만 게임 저작권 문제해...
3     로블록스  진짜 모든 사람에게 추천드려요!\n왜냐하면 모든 장르의 게임이 다있거든요!\n 컴퓨...
4     로블록스       와!로블록스아시는구나!겁나갓겜입니다\n같은 이미지가 있어서 그렇지 생각보단 ㄱㅊ
..     ...                                                ...
687  테일즈런너  얼라시절 정말 열심히 했었다\n그때쯤 유키가 신캐였던가 그랬지\n난 컨트롤이 그때도...
688  테일즈런너  누구는 추억 보정 잔뜩 들어간 게임이라고 말하지만\n그 당시 이 게임이 가진 매력과...
689  테일즈런너                                             재밌는 테런
690  테일즈런너                                                재밋다
691  테일즈런너               똥망겜이라고 하지만 누구보다 열심히 플레이 했던\n 어린 나...

[692 rows x 2 columns]


## 형태소 분석기를 이용하여 리뷰의 단어 추출

In [7]:
import pandas as pd

# 데이터 불러오기기
data_review = pd.read_csv('./datas/game_reviews.csv')

data_review

,Game,Review
0,로블록스,이 게임 하나의 모든 장르에 게임이 있어서 정말 추천함
1,로블록스,진짜 할 거 없어요
2,로블록스,게임이라기 보단 게임들이 모여있는 플렛폼에 더 가깝다.\n하지만 게임 저작권 문제해...
3,로블록스,진짜 모든 사람에게 추천드려요!\n왜냐하면 모든 장르의 게임이 다있거든요!\n 컴퓨...
4,로블록스,와!로블록스아시는구나!겁나갓겜입니다\n같은 이미지가 있어서 그렇지 생각보단 ㄱㅊ
...,...,...
687,테일즈런너,얼라시절 정말 열심히 했었다\n그때쯤 유키가 신캐였던가 그랬지\n난 컨트롤이 그때도...
688,테일즈런너,누구는 추억 보정 잔뜩 들어간 게임이라고 말하지만\n그 당시 이 게임이 가진 매력과...
689,테일즈런너,재밌는 테런
690,테일즈런너,재밋다


In [2]:
import pandas as pd

# 데이터 불러오기
data = pd.read_csv('../data/game_reviews.csv')
meta = pd.read_csv('../data/game_meta.csv')
meta.dropna(inplace=True)

# 데이터 통합
meta.rename(columns={"name": "Game"}, inplace=True)
data = pd.merge(data, meta, on="Game", how="outer")
data.dropna(inplace=True)
data



,Game,Review,main_age,difficulty,playtime,achievement,genre
0,던전앤파이터,넥슨 너네는 던파 도트팀 사운드팀 진짜 잘해줘야된다,25_29,3.10,79.19,2.27,RPG
1,던전앤파이터,메이플 상위버전 그래픽은 옛날이라구리지만 살아있다는게 신기함.\n여전히 뽑아서 쓰고...,25_29,3.10,79.19,2.27,RPG
2,던전앤파이터,지금의 이미지가 구려서 그렇지 게임은 정말 재미있다. 처음 나왔을 때 던전앤드래곤을...,25_29,3.10,79.19,2.27,RPG
3,던전앤파이터,게임이 어둑섬까지 잘 되다가 안개신 레이드에서 불쾌감을 나타낼만한 환경으로 조성이 ...,25_29,3.10,79.19,2.27,RPG
4,던전앤파이터,뉴비여서 고인물의입장은 잘 모르겠지만\n고인물들이 뉴비유입을 막는건 말이안된다생각\...,25_29,3.10,79.19,2.27,RPG
...,...,...,...,...,...,...,...
1728,피파23,커리어 모드 혁신이 필요함,13_18,3.06,60.00,3.23,스포츠
1729,피파23,쇼케이스때 지성이형 본것만해도 만족\n게임도 만족! 한글화도 감사. 축구게임은 앞으...,13_18,3.06,60.00,3.23,스포츠
1730,피파23,축구팬으로서 솔직히 재미없다고 하면 거짓말이지만\n커리어모드만 하는 나로서는 아쉬운...,13_18,3.06,60.00,3.23,스포츠
1731,피파23,월드컵과\n마지막 EA의 피파,13_18,3.06,60.00,3.23,스포츠


In [3]:
from konlpy.tag import Kkma
from collections import Counter
import re


# 모든 리뷰를 담을 딕셔너리 생성
game_nouns = {}

# Okt 형태소 분석기 초기화
kkma = Kkma()

# 불용어 지정
stopwords = ["때문", "그때", "정말", "매우", "게임", "하다"]

# 주연령별로 리뷰 분석
for age in data['main_age'].unique():
    # 해당 게임의 모든 리뷰 추출
    game_reviews = data[data['main_age'] == age]['Review']

    # 모든 리뷰에서 명사 추출
    nouns_list = []
    for review in game_reviews:
        if isinstance(review, str):
            review_edit = re.sub("[^0-9a-zA-Z가-힣\\s+]", "", review)
            review_edit_pos = kkma.pos(review_edit)
            sub_list = []
            for word, pos in review_edit_pos:
                if len(word) == 1 or word in stopwords:
                    continue
                if pos.startswith('N'):  # 태그가 N으로 시작하는 모든 명사 추출
                    sub_list.append(word)
            nouns_list.extend(sub_list)
    
    # 게임별 명사 리스트 저장
    game_nouns[age] = nouns_list

    # 그래픽 관련 추가할 명사 : 그래픽, 화면, 화질, 퀄리티, 비디오, 모션
    # 명사+동사 리뷰도 뽑기
    

    print(f"{age} 의 리뷰의 등장하는 명사> ")
    print(set(nouns_list))
    print("")

print(game_nouns)

25_29 의 리뷰의 등장하는 명사> 
{'위치', '재미', '찌개', '그것', '각성', '근면', '폭파', '문화', '노가다', '당시', '균열', '성적', '조종', '펜타', '군인', '수다', '주술사', '의문', '승률', '저녁', '이지스', '좀비', '오해', '마우스', '타락', '사악', '따름', '양털', '족제비', '방향', '충성', '음미', '초보', '전역', '2006', '타로', '상납', '벌판', '수행', '성인', '공략', '피해', '모험', '마수', '피시', '저거', '분노', '공장', '옛날', '이전', '복각', '등록', '우리나라', '잡생각', '보안', '메이', '인방', '개최', '적극', '보스', '구매', '디자인', '선물', '잘만', '내의', '라이', '양키', '말세', '노력', '개판', '가입', '소실', '소리', '고생', '지게', '완벽', '존재', '잡담', '리프트', '아빠', '아레나', '하지', '드리프트', '넥센', '영진', '그분', '소가', '시스템', '호환성', '누군가', '완성도', '타지', '반응', '악기', '여타', '스팀', '80', '현역', '캐릭터', '자신감', '형식', '한번', '컨트롤', '이래', '82', '실패', '이랑', '여유', '시분', '저기', '밤새', '결국', '다라', '변화', '간만', '용기', '독보적', '승부', '평점', '인플레이', '요약', '주심', '고향', '로스트', '한계', '달성', '항목', '나이트', '독성', '술자리', '주려', '고등학교', '핸드폰', '총성', '자신', '명함', '우의', '이득', '오버', '전략', '지장', '멀티', '선택지', '최초', '충전', '상당량', '이것저것', '코드', '공격대', '아크', '눈물', '퀄리티', '찰흙', '도착', '조정', '시국', '30'

In [24]:
# word cloud
from wordcloud import WordCloud
from collections import Counter
from PIL import Image 
import numpy as np 
import matplotlib.pyplot as plt
import koreanize_matplotlib 

# 한글 폰트 설정 (맥OS 기준, 윈도우의 경우 적절한 폰트 경로로 변경 필요)
# font_path = 'C:/Windows/Fonts/HMFMOLD.TTF'
font_path = "/System/Library/Fonts/Supplemental/AppleSDGothicNeo.ttc"

# 각 게임별로 워드클라우드 생성
for game, nouns in game_nouns.items():
    # 단어 빈도수 계산
    word_counts = Counter(nouns)

    # 마스크 이미지 가져오기
    # mask_img = np.array(Image.open("./images/console.png"))
    
    # 워드클라우드 생성
    wordcloud = WordCloud(
        font_path=font_path,
        width=512, 
        height=512,
        # mask=mask_img,
        background_color='white',
        colormap="twilight"
    ).generate_from_frequencies(word_counts)
    
    # 워드클라우드 그리기
    plt.figure(figsize=(16, 16))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"{game} 리뷰 워드클라우드 분석")
    plt.show()
    
    # 이미지 저장
    # plt.savefig(f"{game}_wordcloud.png")
    # plt.close()

print("모든 게임의 워드클라우드가 생성되었습니다.")

모든 게임의 워드클라우드가 생성되었습니다.


In [16]:
game_nouns.keys()

dict_keys(['25_29', '06_12', '19_24', '13_18'])

# TODO

In [ ]:
# apriori
from apyori import apriori

# 데이터 불러오기
data = pd.read_csv('./data/game_reviews.csv')

game_reviews = data[data['Game'] == game]['Review']

result = list(apriori(game_reviews))
data = pd.DataFrame(result)
data

# result = list(apriori(game_nouns_values))
# print(apriori(game_nouns_values))
# data = pd.DataFrame(result)
# data

## 그래픽에 대한 리뷰 추출

## 분위기에 대한 리뷰 추출

# 게임별 리뷰의 워드클라우드